In [70]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
import pandas as pd
import random

torch.manual_seed(100)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(True)

In [71]:
df = pd.read_excel('pashto_tags_data.xlsx', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,دومره,واوره,نه,وه,شوې,0,0,0,0,0,0
1,adv,noun,adv,hvb,verb,0,0,0,0,0,0
2,واوره,په,کابل,وریږی,خوشاله,پاکستان,دے,0,0,0,0
3,noun,pre,noun,verb,adj,noun,hvb,0,0,0,0
4,د,کابل,په,سیمو,واوره,وریږی,خلک,ټول,خوشهاله,دی,0


In [72]:

START_TAG = "<START>"
STOP_TAG = "<STOP>"

# EMBEDDING_DIM = 100
# HIDDEN_DIM = 100

EMBEDDING_DIM = 300
HIDDEN_DIM = 300

In [73]:

values = df.values
training_data = []

for i in range(0, len(df)-1, 2):
  sentence = values[i]
  tags = values[i+1]
  training_data.append((sentence, tags))

In [74]:

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)


tag_to_ix = {}
for sentence, tags in training_data:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


tag_to_ix[START_TAG]=len(tag_to_ix)
tag_to_ix[STOP_TAG]=len(tag_to_ix)


idx_to_tag = {k:j for j, k in tag_to_ix.items()}

In [75]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [76]:

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),  torch.randn(2, 1, self.hidden_dim // 2) )

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)

                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [77]:
factor = int(len(training_data) * 0.8)
trainingData = training_data[:factor]
testData = training_data[factor:]

In [78]:
epochs  =  20


model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.Adagrad(model.parameters(), lr=0.1, weight_decay=1e-4)

totalWords = np.array([len(i[0]) for i in training_data]).sum()

# Make sure prepare_sequence from earlier in the LSTM section is loaded
print("training on ", len(trainingData), "sentences  and ", totalWords, " words")

for epoch in range(epochs):
    model.train()
    loss_ = []
    for sentence, tags in trainingData:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss_.append(loss.item())
        loss.backward()
        optimizer.step()
    loss_ = np.array(loss_).mean()
    print(f"Epochs {epoch+1} / {epochs}: loss {loss_}")

training on  176 sentences  and  2431  words
Epochs 1 / 20: loss 9.084541591730984
Epochs 2 / 20: loss 1.9319062232971191
Epochs 3 / 20: loss 0.5078650821339
Epochs 4 / 20: loss 0.2379165996204723
Epochs 5 / 20: loss 0.09289277683604848
Epochs 6 / 20: loss 0.056041847575794564
Epochs 7 / 20: loss 0.03406537662852894
Epochs 8 / 20: loss 0.02354075691916726
Epochs 9 / 20: loss 0.02358466928655451
Epochs 10 / 20: loss 0.019605679945512253
Epochs 11 / 20: loss 0.018677928230979225
Epochs 12 / 20: loss 0.01690231670032848
Epochs 13 / 20: loss 0.01762624220414595
Epochs 14 / 20: loss 0.015241622924804688
Epochs 15 / 20: loss 0.013219053095037287
Epochs 16 / 20: loss 0.014519041234796698
Epochs 17 / 20: loss 0.013236869465221058
Epochs 18 / 20: loss 0.012272444638338957
Epochs 19 / 20: loss 0.011877580122514204
Epochs 20 / 20: loss 0.012113787911155006


In [79]:
correct = 0
mistak = 0
total=0

testWords = np.array([len(i[0]) for i in testData]).sum()

print("testting on ", len(testData), "sentences and ", testWords, " words")

predictions = []

with torch.no_grad():
    for sentence, gold_tags in testData:
        precheck_sent = prepare_sequence(sentence, word_to_ix)
        output = model(precheck_sent)
        pred_tags = [idx_to_tag[i] for i in output[1]]
        # predictions.append((sentence, gold_tags, pred_tags))
        predictions.append(sentence)
        predictions.append(gold_tags)
        predictions.append(pred_tags)
        predictions.append(["*" for i in range(len(pred_tags))])
        for t1 , t2 in zip(gold_tags,  pred_tags):
            if(t1==t2):
                correct+=1
            else:
                mistak+=1
            total+=1

print("correct=", correct, "/" , total,    "mistakes = ", mistak, "/", total)
print("Accuracy =", correct/total)

testting on  45 sentences and  495  words
correct= 403 / 495 mistakes =  92 / 495
Accuracy = 0.8141414141414142


In [80]:
results = pd.DataFrame(predictions, columns = ['word_'+str(i+1) for i in range(len(predictions[0]))])
results.to_excel('results.xlsx', index=False)

In [81]:
results = results.dropna()

In [82]:
def generate_report(results):
    html= "<html>\n"
    html+= """  <style>
                  th, td {
                      border-radius: 10px;
                      background-color:#D3D3D3;
                  }
                </style> \n """

    html+= "<body>\n"
    eng = [i for i in 'abcdefghijklmnopqrstuvwxyz']
    html+="<table>"
    for i, row in enumerate(results.values):
      row  = row[::-1]
      html+= "\n\t<tr>\n"
      for j in row:
        if str(j)=='0':
          continue
        if str(j)=='nan':
           html+= "\t<td> <p> . </p> </td>"
        else:
           html+= "\t<td>" + str(j) + "</td>"
      html+="\n\t</tr>\n"

    html+="</table>"
    html+= "</body>\n"
    html+= "</html>\n"

    with open('report.html', 'w', encoding='utf-8') as report:
      report.writelines(html)
    print("done")

In [84]:
generate_report(results)

done
